# RF Chain and Channelisation Commissioning Test

## Table of Contents\n
#### [0.1 - Global Configuration](#01---global-configuration)
#### [0.2 - Setup Test Equipment](#02---setup-test-equipment)
#### [1 -> Test Steps](#1---test-steps)

## 0.1 Global Configuration

### 0.1.1 Import dependencies

In [2]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 0.1.2 Setup Global Variables and Configuration

In [3]:
debug_mode = True  # This setting enables printing of diagnostics
enable_logging = True  # This enables logging and sets the global log_level to debug
dishlmc_enabled = True  # Set this to true if you have a dish LMC deployment
executon_environment = Environment.Integration
branch_name = "none"  # Set this if you are using an on-demand deployment (i.e. Environment.CI)
if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
# namespace_override parameter can be used to override auto-configured SUT namespace
sut_namespace_override = ""
subarray_count = 1
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
print(f"SUT configured: {str(sut)}")
dish_ids = ["001", "036"]
# namespace_override parameter can be used to override auto-configured dish namespace
dish_namespace_overrides = ["", ""]
dish_deployments: List[TangoDishDeployment] = []
if dishlmc_enabled:
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured: {dish}")
        dish_deployments.append(dish)

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

Test Equipment Configured: TangoTestEquipment{namespace=test-equipment; tango_host=tango-databaseds.test-equipment.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.test-equipment.svc.miditf.internal.skao.int:8765}
SUT configured: TangoSUTDeployment{subarray_index=1; namespace=integration; tango_host=tango-databaseds.integration.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration.svc.miditf.internal.skao.int:8765}
Dish 001 configured: TangoDishDeployment{dish_id=ska001; namespace=integration-dish-lmc-ska001; tango_host=tango-databaseds.integration-dish-lmc-ska001.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration-dish-lmc-ska001.svc.miditf.internal.skao.int:8765}
Dish 036 configured: TangoDishDeployment{dish_id=ska036; namespace=integration-dish-lmc-ska036; tango_host=tango-databaseds.integr

### 0.1.3 Test Connections to Namespaces

In [ ]:
sut.smoke_test()
test_equipment.smoke_test()
for dish_deployment in dish_deployments:
    dish_deployment.smoke_test()

### 0.1.4 Export System Configuration

In [ ]:
deployment: TangoDeployment
for deployment in [sut, test_equipment, *dish_deployments]:
    deployment.export_chart_configuration(output_dir=notebook_output_dir)

## 0.2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 0.2.1 Configure Test Equipment State 

In [ ]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

### 0.2.2 Print Test Equipment Diagnostics

In [ ]:
test_equipment.print_diagnostics()

### 0.2.3 Create Test Equipment Plot

In [ ]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

### 0.2.4 Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

### 0.2.5 Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

## 1 Test Steps 

**Step 1:**

Set CW generator to approximately 800MHz (freq1) & -30dBm (power1), and wideband noise to approximately -132dBm/Hz at the SPF Band 1 Receiver inputs. ie. Band 1 output on and attenuation approx 10dB.  

Expected Result:  
Test equipment configured

In [ ]:
# test equipment setup values
frequency_to_set = 800.0002e6
power_level = -30.0
attenuation = 10
band = 1

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(3)
# print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == power_level, print(
    f"Power level required is {power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

In [ ]:
# not directly part of test but useful lines
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator
print(prog_atten.get_attribute_list())
print(SSC.get_attribute_list())

**Step 2:**

Assign resources, configure the system for a subarray to run an observation scan in Band 1, with correlator averaging set to no more than 2 seconds.  

Expected Result:  
SUT Configured

This section runs steps 3.1 to 3.9 of the Configure Scan Tests

In [ ]:
# need to figure out where / how to set correlator averaging time - asked on Help TMC.
# update. Apparently it is set to 0.14 seconds

Use OSO scripting interface to TMC to run the script for assigning resources for a single sub-array, configuring and running a  scan.  

### Setup Telescope Monitoring

#### Configure Telescope Monitoring

In [7]:
# setup monitoring
# use telescope state object for state monitoring
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
telescope_state = get_telescope_state(device_model, sut)
# use monitor plot as a dashboard
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
# set up events to monitor
telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
telescope_state.subscribe_to_subarray_resource_state(
    telescope_monitor_plot.observe_subarray_resources_state
)
telescope_state.subscribe_to_subarray_configurational_state(
    telescope_monitor_plot.observe_subarray_configuration_state
)
telescope_state.subscribe_to_subarray_scanning_state(
    telescope_monitor_plot.observe_subarray_scanning_state
)
output_notebook()

ConnectionFailed: DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_NoUsableProfile
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/cppapi/client/devapi_base.cpp:604)
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Failed to connect to database on host tango-databaseds.integration.svc.miditf.internal.skao.int with port 10000
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/cppapi/client/devapi_base.cpp:604)
  reason = API_CantConnectToDatabase
severity = ERR]
]

#### Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [ ]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)

### Print System Diagnostics

#### Print TMC Diagnostics

In [5]:
sut.print_tmc_diagnostics()

ConnectionFailed: DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_NoUsableProfile
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/cppapi/client/devapi_base.cpp:604)
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Failed to connect to database on host tango-databaseds.integration.svc.miditf.internal.skao.int with port 10000
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/cppapi/client/devapi_base.cpp:604)
  reason = API_CantConnectToDatabase
severity = ERR]
]

#### Print CSP-LMC Diagnostics

In [ ]:
sut.print_csp_diagnostics()

#### Print CBF Diagnostics

In [ ]:
sut.print_cbf_diagnostics()

#### Print SDP Diagnostics

In [ ]:
sut.print_sdp_diagnostics()

#### Print Dish-LMC Diagnostics

In [ ]:
for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id} - {dish_deployment.namespace}: Diagnostics")

    dish_deployment.print_diagnostics()

#### Print Full System Diagnostics

In [ ]:
print("SUT: Full Diagnostics")
sut.print_full_diagnostics()

for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id}: Full Diagnostics")
    dish_deployment.print_full_diagnostics()

print("Test Equipment: Full Diagnostics")
test_equipment.print_full_diagnostics()

### Setup ODA

In [ ]:
os.environ["ODA_URI"] = (
    "http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/"
)
eb_id = oda_helper.create_eb()
print(f"Execution Block ID: {eb_id}")

### Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [ ]:
sub = SubArray(subarray_id)
tel = Telescope()

### Load VCC Configuration in TMC

In [ ]:
# This should only be executed for a fresh deployment (i.e. Telescope is OFF.
# If you have restarted the subarray, you should not run this command
sut.load_dish_vcc_config()

### Turn telescope ON

In [ ]:
# set to ON only if OFF
# If you have restarted the subarray, you should not run this command (Telescope is already ON)
# dish_lmc mode must be in LP_standby and TMC VCC before trying to turn the telescope ON
# Takes about 1m20s
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    tel.on()
else:
    assert (
        telescope_monitor_plot.on_off_state == "ON"
    ), f"Cant continue with telescope in {telescope_monitor_plot.on_off_state}"

### 3.7 Observation Definition

#### 3.7.1 Create the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [ ]:
dish_ids = [d.dish_id.upper() for d in dish_deployments]
default_target_specs = get_default_target_specs_sb(dish_ids)
observation = ObservationSB(target_specs=default_target_specs)

target_specs = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_1,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in target_specs.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(target_specs)

for target_id, target in target_specs.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_def_id = "flux calibrator"
observation.add_scan_sequence([scan_def_id])

#### 3.7.2 Mid configuration schema input used by observing commands

[Configuration Schemas-OET→TMC(Mid)](https://developer.skao.int/projects/ska-telmodel/en/latest/)

In [ ]:
telescope_monitor_plot.show()

#### 3.7.3 Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [ ]:
observation.eb_id = eb_id
pdm_allocation = observation.generate_pdm_object_for_sbd_save(target_specs)

sbd = oda_helper.save(pdm_allocation)
sbd_id = sbd.sbd_id
pdm_allocation.sbd_id = sbd_id
print(f"Saved Scheduling Block Definition Instance in ODA: SBD_ID={sbd_id}")

### 3.8 Assign Resources
Assign the requested resources to a Subarray

In [ ]:
assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object

if debug_mode:
    request_json = get_request_json(assign_request, AssignResourcesRequest, True)
    print("AssignResourcesRequest:", json.dumps(json.loads(request_json), indent=2))

sub.assign_from_cdm(assign_request, timeout=120)

In [ ]:
telescope_monitor_plot.show()

### 3.9 Configure Scan
Configure the telescope  on first target in sequence - may be modified to configure and run multiple targets at a later time.

In [ ]:
configure_object = observation.generate_scan_config_sb(
    pdm_observation_request=pdm_allocation,
    scan_definition_id=scan_def_id,
    scan_duration=10.0,
).as_object

if debug_mode:
    cfg_json = get_request_json(configure_object, ConfigureRequest)
    print(f"ConfigureRequest={cfg_json}")

sub.configure_from_cdm(configure_object, timeout=120)
time.sleep(2)

In [ ]:
telescope_monitor_plot.show()

**Step 3:**

Start scan and signal path recording and monitor the SDP Spectrometer Display.  

Expected Result:  
_CW + noise in SDP Spectrometer Display_
 
This section runs step 3.10 of the Configure Scan Tests and provides a link to the SDP signal displays

### 3.10 Run the Scan

In [ ]:
sub.scan(timeout=120)
telescope_monitor_plot.show()

In [ ]:
print(sut.signal_displays_endpoint)

**Step 4:**

~10 seconds after recording has started, use Test Control Script to adjust CW generator signal level by -3dB. 

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

**Note: This and following steps that change the test equipment settings during the SCAN are run from the rf-chain_and_channelisation_test_equipment notebook but included here for completion**

**Step 5:**

After a further ~10 seconds, use Test Control Script to adjust CW generator signal level by -3dB.  

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

**Step 6:**

After a further ~10seconds, use Test Control Script to increase the CW generator signal frequency by 10MHz.  

Expected Result:  
*CW frequency changes by 10MHz in SDP Spectrometer Display*

**Step 7:**

AAfter a further ~10seconds, end scan and recording. Monitor feedback from SDP storage. 

Expected Result:  
*SDP storage listing includes new dataset*

**Step 8:**

Retrieve the dataset from the SDP storage (using Data Product Dashboard).  

Expected Result:  
_Copy of data file successfully downloaded_

**Step 9:** 

ANALYSIS:  

Analysis steps not done in Notebook

**Step 10:**

Determine the receiver band of the dataset.  

Expected Result:  
_Band 1_

**Step 11:**

Determine the number of frequency channels recorded.  

Expected Result:  
_Number of frequency channels in the range [51180_

**Step 12:**

Process the data file to identify the dominant CW signal at each time interval. Extract the detected frequency and power level as a time series.  

Expected Result:  
_Record of CW frequency & power for each time interval_

**Step 13:**

Confirm that the time series agrees with the programmed sequence of attenuation and frequency offset steps.
Note: this test is not intended to verify accuracy of frequency & power, so rough agreement is adequate.

Expected Result:  
*1st ~10sec segment: freq1, power1*

*2nd ~10sec segment: freq1, power2*

*3rd ~10sec segment: freq1, power3*

*4th ~10sec segment: freq1+10MHz, power3*

**Step 14:**

  

Expected Result:  
__

**Step 15:**

Set CW generator to approximately 1500MHz (freq2) & -30dBm, and wideband noise to approximately -132dBm/Hz at the SPF Band 2 Receiver inputs.  ie. Band 2 output on and attenuation approx 10dB.

Expected Result:  
*Test equipment configured*

In [ ]:
# test equipment setup values - band 1
frequency_to_set = 1500.0002e6
power_level = -30.0
attenuation = 10
band = 2

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

time.sleep(3)
# print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == power_level, print(
    f"Power level required is {power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)
signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
time.sleep(3)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
print(f"Updated signal generator power level: {signal_generator.power_dbm}")
assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
prog_atten = test_equipment.programmable_attenuator
print(prog_atten.get_attribute_list())
print(
    f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}"
)  # print current attenuationprog_atten.write_attribute('channel_1', 10)

prog_atten.write_attribute("channel_1", attenuation)
time.sleep(1)
print(
    f"Updated Programmable attenuator attenuation : {prog_atten.channel_1}"
)  # print updated attenuation"frequency_to_set = 800e6

**Step 16:**

Assign resources, configure the system for a subarray to run an observation scan in Band 2, with correlator averaging set to no more than 2 seconds.

Expected Result:  
*SUT configured*

Clear scan configuration 

In [ ]:
sub.end()
telescope_monitor_plot.show()

Release Subarray resources

In [ ]:
sub.release()
telescope_monitor_plot.show()

##### Reset the Telescope (On Failure)

Reset the Subarray

In [ ]:
sub.abort()
time.sleep(3)
sub.restart()

Reset the Dish

In [ ]:
dish_deployment: TangoDishDeployment
for dish_deployment in dish_deployments:
    dish_deployment.reset_dish()

Update the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [ ]:
dish_ids = [d.dish_id.upper() for d in dish_deployments]
default_target_specs = get_default_target_specs_sb(dish_ids)
observation = ObservationSB(target_specs=default_target_specs)

target_specs = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in target_specs.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(target_specs)

for target_id, target in target_specs.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_def_id = "flux calibrator"
observation.add_scan_sequence([scan_def_id])

In [ ]:
telescope_monitor_plot.show()

Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [ ]:
observation.eb_id = eb_id
pdm_allocation = observation.generate_pdm_object_for_sbd_save(target_specs)

sbd = oda_helper.save(pdm_allocation)
sbd_id = sbd.sbd_id
pdm_allocation.sbd_id = sbd_id
print(f"Saved Scheduling Block Definition Instance in ODA: SBD_ID={sbd_id}")

Assign the requested resources to a Subarray

In [ ]:
assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object

if debug_mode:
    request_json = get_request_json(assign_request, AssignResourcesRequest, True)
    print("AssignResourcesRequest:", json.dumps(json.loads(request_json), indent=2))

sub.assign_from_cdm(assign_request, timeout=120)

In [ ]:
telescope_monitor_plot.show()

Configure the telescope  on first target in sequence - may be modified to configure and run multiple targets at a later time.

In [ ]:
configure_object = observation.generate_scan_config_sb(
    pdm_observation_request=pdm_allocation,
    scan_definition_id=scan_def_id,
    scan_duration=10.0,
).as_object

if debug_mode:
    cfg_json = get_request_json(configure_object, ConfigureRequest)
    print(f"ConfigureRequest={cfg_json}")

sub.configure_from_cdm(configure_object, timeout=120)
time.sleep(2)

**Step 17:**

Start scan and start signal path recording and monitor the Spectrometer Display.  

Expected Result:  
_CW + noise in SDP Spectrometer Display_

Run the Scan

In [ ]:
sub.scan(timeout=120)
telescope_monitor_plot.show()

**Step 18:**

After ~10seconds, end scan and recording. Monitor feedback from SDP storage .

Expected Result:  
*SDP storage listing includes new dataset*

note: configure command has 10 seconds for scan duration

**Step 19:**

Retrieve the dataset from the SDP storage (using Data Product Dashboard).  

Expected Result:  
_Copy of data file successfully downloaded_

**Step 20:**

ANALYSIS:  

Expected Result:  
__

**Step 21:**

Determine the number of frequency channels recorded.  

Expected Result:  
_Number of frequency channels in the range [51180, 65536] (inclusive). Note that only approximately 16000 channels are expected to be available prior to AA3_

**Step 22:**

Determine the receiver band of the dataset.  

Expected Result:  
_Band 2_

**Step 23:**

Process the data file to identify the dominant CW signal at each time interval. Extract the detected frequency and power level as a time series.  

Expected Result:  
_Record of CW frequency & power for each time interval_

**Step 24:**

Confirm that the dominant CW signal frequency agrees with the programmed signal frequency.

Note: this test is not intended to verify accuracy of frequency & power, so rough agreement is adequate. 

Expected Result:  
_freq2_

### 3.11 Post Observation teardown
If the observation executed successfully, you can use the following commands to reset the telescope.

#### 3.11.1 Clear scan configuration 

In [ ]:
sub.end()
telescope_monitor_plot.show()

#### 3.11.2 Release Subarray resources

In [ ]:
sub.release()
telescope_monitor_plot.show()

### 3.12 Reset the Telescope (On Failure)
Set booleans to True to reset the system after a failed execution.

#### 3.12.1 Reset the Subarray

In [ ]:
sub.abort()
time.sleep(3)
sub.restart()

#### 3.12.2 Reset the Dish

In [ ]:
dish_deployment: TangoDishDeployment
for dish_deployment in dish_deployments:
    dish_deployment.reset_dish()

## 4 Debug Telescope

### 4.1 Display Telescope State

In [ ]:
telescope_state.state

### 4.2 Display Telescope State Subscriptions

In [ ]:
telescope_state.state_monitor.subscriptions

### 4.3 Display Dish LMC State

In [ ]:
dishes_to_debug = [d.dish_id for d in dish_deployments]
for dish in dish_deployments:
    if dish.dish_id in dishes_to_debug:
        dish.print_diagnostics()

### 4.4 Display CSP LMC State

In [ ]:
sut.print_csp_diagnostics()

### 4.5 Display TMC State

In [ ]:
sut.print_tmc_diagnostics()

### 4.6 Display SDP State

In [ ]:
sut.print_sdp_diagnostics()

## 5 Dashboard Links

### 5.1 Taranta

In [ ]:
print(f"SUT: {sut.taranta_endpoint}")
print(f"Test Equipment: {test_equipment.taranta_endpoint}")
for dish in dish_deployments:
    print(f"Dish LMC {dish.dish_id}: {dish.taranta_endpoint}")

### 5.2 Data Product Dashboard

https://k8s.miditf.internal.skao.int/ska-dpd/dashboard/

### 5.3 SDP Signal Displays

In [ ]:
print(sut.signal_displays_endpoint)